In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
sys.path.append('../src')
from torch import nn
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns=100
from multiprocessing import Pool,cpu_count
from sklearn.metrics import pairwise_distances

In [2]:
# import librosa
# import torchaudio

In [2]:
from datalib import VentilatorDataRegression
import modellib
from utils import fc
import datalib
from model import LinBnReLu

In [3]:
DATA_DIR = '/mnt/disks/extra_data/kaggle/ventilator_prediction/'
R_MAP = {5: 0, 50: 1, 20: 2}
C_MAP = {20: 0, 50: 1, 10: 2}

In [4]:
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
train.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))
test.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))

In [10]:
m = sorted(train.pressure.unique().tolist())
m[0]-m[1]

-0.07030214545121005

In [10]:
# train.pressure.value_counts(1).hist()

In [5]:
train = fc(train)

5
10
5
10


In [6]:
train.R = train.R.map(R_MAP)
train.C = train.C.map(C_MAP)

In [7]:
group_dict={}
for i,x in tqdm(train.groupby('breath_id'),total=train.breath_id.nunique()):
    m = x.to_dict(orient='records')
    group_dict[i] = {k:[a[k] for a in m] for k in [k for k in m[0].keys() if k not in ['id','breath_id']]}

100%|██████████| 75450/75450 [08:50<00:00, 142.24it/s]


In [16]:
# %%time
# train_uin_matrix = get_matrix_dict(train[['breath_id','u_in','u_out', 'time_step']])

In [9]:
config = OmegaConf.load('../configs/lstm-regression.yaml')
# config = OmegaConf.load('../configs/lstm-classification-v2.yaml')
config.model.kwargs["output_dim"] = 1

In [10]:
data = getattr(datalib,config.dataset.train['class'])(**config.dataset.train['kwargs'],group_dict=group_dict)
dl = DataLoader(
    dataset=data,
    shuffle=True,
    num_workers=1,
    batch_size=8,
    pin_memory=True,
)

In [13]:
%%time
m = next(iter(dl))

CPU times: user 5.09 ms, sys: 321 ms, total: 326 ms
Wall time: 604 ms


In [14]:
from litmodellib import Model
# import joblib

In [21]:
# joblib.dump(pressure_reverse_dict,'../sorted_pressure_mapper.pkl')
lit_model = Model(
    config,
    num_train_iter=1,
    num_steps=1,
)

/home/sahil_okcredit_in/miniconda3/envs/torch-py3.8/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


{'init_type': 'yakama'}
LSTMDpReLu(
  (rnn): LSTM(148, 1024, batch_first=True, dropout=0.1, bidirectional=True)
  (dp): Dropout(p=0.1, inplace=False)
)
LSTMDpReLu(
  (rnn): LSTM(2048, 512, batch_first=True, dropout=0.1, bidirectional=True)
  (dp): Dropout(p=0.1, inplace=False)
)
LSTMDpReLu(
  (rnn): LSTM(1024, 256, batch_first=True, dropout=0.1, bidirectional=True)
  (dp): Dropout(p=0.1, inplace=False)
)
LSTMDpReLu(
  (rnn): LSTM(512, 128, batch_first=True, dropout=0.1, bidirectional=True)
  (dp): Dropout(p=0.1, inplace=False)
)


In [18]:
# lit_model.model

In [22]:
a,b = lit_model.model(m)

In [23]:
a.shape,b.shape

(torch.Size([8, 80]), torch.Size([8, 80]))

In [ ]:
# lit_model.model

In [15]:
# joblib.load('../sorted_pressure_mapper.pkl')

In [66]:
# joblib.load('../pressure_mapper.pkl')

In [233]:
a = lit_model.model(m).view(-1,950)

In [234]:
a.shape

torch.Size([640, 950])

In [152]:
from losslib import FocalLoss
loss = FocalLoss(gamma=0,alpha=1)

In [220]:
from losslib import FocalLoss
loss = FocalLoss(gamma=10,alpha=None)

In [221]:
loss2 = FocalLoss2(gamma=10,alpha=1)

In [222]:
loss(a.cpu(),m['target'].view(-1))

tensor(6.7878, grad_fn=<MeanBackward0>)

In [223]:
loss2(a.cpu(),m['target'].view(-1))

tensor(6.7878, grad_fn=<MeanBackward0>)

In [160]:
targets = m['target'].view(-1)
targets = F.one_hot(targets,950)

In [170]:
x = nn.Softmax(1)(a.cpu())

In [184]:
import random
x = torch.rand(12800,4)*random.randint(1,10)
x = Variable(x.cuda())
l = torch.rand(12800).ge(0.1).long()
l = Variable(l.cuda())

In [185]:
x.shape,l.shape

(torch.Size([12800, 4]), torch.Size([12800]))

In [192]:
(torch.rand(128,8,4)*1000)

torch.Size([128, 8, 4])

In [186]:
loss(x,l)

tensor(0.1883, device='cuda:0')

In [ ]:
loss(a,m['target'].view(-1))

In [125]:
loss(a,m['target'].view(-1))

tensor(6.8511, grad_fn=<MeanBackward0>)

In [155]:
# loss(a.cpu(),m['target'].view(-1))

In [126]:
from litmodellib import smoothing_loss

In [130]:
import losslib

In [149]:
a = a.cuda(2)

In [148]:
loss.alpha.type_as(a)

tensor([1., 0.], device='cuda:0')

In [151]:
loss.alpha.type_as(a.data)

tensor([1., 0.], device='cuda:2')

In [138]:
smoothing_loss(getattr(losslib, config.loss["class"])(**config.loss["kwargs"]),a,m['target'].view(-1))

tensor(20.5830, grad_fn=<AddBackward0>)

In [115]:
alpha = 1
gamma = 0

In [116]:
targets = m['target'].view(-1)
targets = F.one_hot(targets,950)
pt = nn.Softmax(dim=1)(a)
logpt = torch.log(pt)
alpha = torch.tensor([alpha, 1-alpha])
at = alpha.gather(0, targets.data.view(-1))

In [117]:
at.shape,pt.shape,logpt.shape

(torch.Size([608000]), torch.Size([640, 950]), torch.Size([640, 950]))

tensor(6.8511, grad_fn=<MeanBackward0>)

In [119]:
nn.CrossEntropyLoss()(a,m['target'].view(-1))

tensor(6.8595, grad_fn=<NllLossBackward>)

In [ ]:
def focal_loss()

In [42]:
torch.matmul(b.permute(0,2,1),a).permute(0,2,1).shape

torch.Size([8, 80, 1024])

In [35]:
b.squeeze(1).shape

torch.Size([8, 80, 80])

In [127]:
# import model
# class LSTMAttnClassification(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.config = config
#         emb_layers = []
#         for k in self.config.embedding_layer:
#             emb_layers.append(nn.Embedding(**self.config.embedding_layer[k]))
#         self.embedding = nn.Sequential(*emb_layers)

#         seq_layers = []
#         for k in self.config.rnn_layer:
#             seq_layers.append(
#                 getattr(nn, self.config.rnn_layer[k]["class"])(
#                     **self.config.rnn_layer[k]["kwargs"]
#                 )
#             )
#         self.rnn = nn.Sequential(*seq_layers)
        
#         self.fc = nn.Sequential(
#             *[
#                 LinBnReLu(
#                     in_dim=2 * 512,
#                     out_dim=512,
#                     is_bn=False,
#                     dropout=0.1,
#                 ),
#                 LinBnReLu(
#                     in_dim=512, out_dim=self.config.output_dim, is_bn=False, dropout=0
#                 ),
#             ]
#         )
#         wt_init = getattr(model, self.config.rnn_init["class"])(
#             **self.config.rnn_init["kwargs"]
#         )
#         print(wt_init.__dict__)
#         for layer in self.rnn:
#             wt_init(layer)

#     def forward(self, x):
#         x_cat, x_num = x["cat"], x["num"]
#         attn_matrix = x['u_in_matrix']
#         x_cat = torch.cat(
#             [self.embedding[i](x_cat[:, :, i]) for i in range(len(self.embedding))],
#             dim=-1,
#         )
#         x_in = torch.cat([x_num, x_cat], dim=-1)
        
#         for layer in self.rnn:
#             x_in, _ = layer(x_in)
#             wt_x_in = torch.matmul(x_in.permute(0,2,1),attn_matrix.float()).permute(0,2,1)
#             x_in = x_in+wt_x_in        
#         output = self.fc(x_in)
#         return output

In [22]:
mod = LSTMAttnClassification(config.model['kwargs'])

NameError: name 'LSTMAttnClassification' is not defined

In [124]:
mod

LSTMAttnClassification(
  (embedding): Sequential(
    (0): Embedding(2, 32)
    (1): Embedding(3, 32)
    (2): Embedding(3, 32)
  )
  (rnn): Sequential(
    (0): LSTM(148, 512, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (1): LSTM(1024, 512, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc): Sequential(
    (0): LinBnReLu(
      (dropout): Dropout(p=0.1, inplace=False)
      (act): ReLU()
      (lin): Linear(in_features=1024, out_features=512, bias=True)
    )
    (1): LinBnReLu(
      (dropout): Dropout(p=0, inplace=False)
      (act): ReLU()
      (lin): Linear(in_features=512, out_features=950, bias=True)
    )
  )
)

In [125]:
preds = mod(m)

In [66]:
# nn.Conv2d(in_channels=1,out_channels=16,)

In [54]:
preds.permute(0,2,1).shape

torch.Size([8, 256, 80])

In [68]:
torch.matmul(preds.float(),m['u_in_matrix'].float()).shape

torch.Size([8, 256, 80])

In [29]:
preds = preds.view(-1,950)

In [30]:
preds.shape

torch.Size([640, 950])

In [34]:
target = m['target'].view(-1)

In [61]:
%%time
nn.CrossEntropyLoss()(preds,target)

CPU times: user 1.81 ms, sys: 75 µs, total: 1.88 ms
Wall time: 1.55 ms


tensor(6.8581, grad_fn=<NllLossBackward>)

In [62]:
def loss_fn(y_pred, y_true):
    criterion = nn.CrossEntropyLoss()

    loss = criterion(y_pred, y_true)

    for lag, w in [(1, 8/15), (2, 4/15), (3, 2/15), (4, 1/15)]:
        neg_lag_target = nn.ReLU()(y_true - lag)
        neg_lag_target = neg_lag_target.long()
        neg_lag_loss = criterion(y_pred, neg_lag_target)
        pos_lag_target = 949 - nn.ReLU()((949 - (y_true + lag)))
        pos_lag_target = pos_lag_target.long()
        pos_lag_loss = criterion(y_pred, pos_lag_target)
        loss += (neg_lag_loss + pos_lag_loss) * w
    return loss

In [63]:
%%time
loss_fn(preds,target)

CPU times: user 114 ms, sys: 7.77 ms, total: 122 ms
Wall time: 10.9 ms


tensor(20.5677, grad_fn=<AddBackward0>)

In [21]:
m1 = torch.stack([preds]*4)
m2 = torch.stack([m['target'].view(-1)]*4)

In [22]:
m1.shape,m2.shape

(torch.Size([4, 640, 950]), torch.Size([4, 640]))

In [24]:
torch.vstack([x for x in m1]).shape

torch.Size([2560, 950])

In [25]:
torch.hstack([x for x in m2]).shape

torch.Size([2560])

In [68]:
# m1 = torch.cat(m1,dim=0)
m2 = torch.cat(m2,dim=0)

In [36]:
def get_true_val(mapping,tensor):
    return torch.tensor([mapping[x.item()] for x in tensor])    

In [40]:
%%time
topk = m1.topk(k=3, dim=1).indices
true_preds = (
    get_true_val(mapping, topk.view(-1))
    .view(-1, 3)
    .median(dim=1)
    .values
)


CPU times: user 767 ms, sys: 0 ns, total: 767 ms
Wall time: 364 ms


In [69]:
m2.shape

torch.Size([20480])

In [37]:
import joblib
mapping = joblib.load('../pressure_mapper.pkl')

In [99]:
config.model.kwargs['output_dim'] = 950

In [100]:
config.model.kwargs['output_dim']

950

In [101]:
%%time
k=1
topk = preds.view(-1,950).topk(k=k,dim=1).indices
torch.tensor([torch.median(get_true_val(mapping,x)).item() for x in topk]).mean()

CPU times: user 116 ms, sys: 261 µs, total: 116 ms
Wall time: 24.2 ms


tensor(33.7162)

In [85]:
%%time
_ = get_true_val(mapping,top3.view(-1)).view(-1,3)

CPU times: user 7.75 ms, sys: 3.86 ms, total: 11.6 ms
Wall time: 10.9 ms


In [103]:
%%time
get_true_val(mapping,topk.view(-1)).view(-1,k).median(dim=1).values.mean()

CPU times: user 792 µs, sys: 4.05 ms, total: 4.84 ms
Wall time: 4.35 ms


tensor(33.7162)

In [59]:
%%time
get_true_val(mapping,preds.view(-1,950).argmax(1)).mean()

CPU times: user 12.6 ms, sys: 0 ns, total: 12.6 ms
Wall time: 3.93 ms


tensor(33.7162)

In [37]:
nn.CrossEntropyLoss()(preds.view(-1,950),m['target'].view(-1))

tensor(6.8633, grad_fn=<NllLossBackward>)

In [42]:
def get_true_val(mapping,tensor):
    return torch.tensor([mapping[x.item()] for x in tensor])    

In [62]:
# get_true_val(pressure_reverse_dict,m['target'].view(-1))

In [81]:
true_preds = get_true_val(pressure_reverse_dict,preds.argmax(2).view(-1))

In [84]:
[preds.argmax(2).view(-1).numpy()]

array([851, 143, 906, 168, 906, 906, 869, 757, 906, 906, 906, 906, 843,
       516, 843, 906, 843, 662, 143, 906, 906, 869, 869, 906, 906, 168,
       843, 280, 906, 143, 168, 843, 843, 906, 843, 906, 843, 280, 843,
       906, 280, 168, 869, 906, 906, 143, 906, 843, 711, 168, 280, 869,
       843, 280, 168, 906, 869, 823, 168, 843, 906, 823, 906, 168, 516,
       869, 280, 906, 851, 143, 843, 906, 143, 869, 869, 906, 143, 143,
       906, 280, 143, 168, 143, 906, 168, 906, 906, 906, 906, 906, 843,
       906, 168, 869, 168, 869, 843, 175, 869, 843, 906, 869, 906, 906,
       906, 843, 869, 906, 843, 906, 861, 906, 906, 168, 869, 168, 843,
       869, 843, 906, 843, 143, 168, 843, 843, 843, 168, 659, 869, 143,
       906, 869, 143, 859, 280, 143, 906, 906, 869, 168, 843, 906, 843,
       906, 143, 168, 869, 143, 843, 143, 143, 843, 143, 843, 906, 869,
       823, 843, 843, 143, 364, 143, 168, 906, 168, 906, 143, 843, 906,
       168, 843, 851, 869, 906, 168, 851, 843, 662, 168, 906, 17

In [ ]:
get_true_val(pressure_reverse_dict,preds.argmax(2).view(-1))

In [86]:
preds.view(-1,950).argmax(1)

tensor([851, 143, 906, 168, 906, 906, 869, 757, 906, 906, 906, 906, 843, 516,
        843, 906, 843, 662, 143, 906, 906, 869, 869, 906, 906, 168, 843, 280,
        906, 143, 168, 843, 843, 906, 843, 906, 843, 280, 843, 906, 280, 168,
        869, 906, 906, 143, 906, 843, 711, 168, 280, 869, 843, 280, 168, 906,
        869, 823, 168, 843, 906, 823, 906, 168, 516, 869, 280, 906, 851, 143,
        843, 906, 143, 869, 869, 906, 143, 143, 906, 280, 143, 168, 143, 906,
        168, 906, 906, 906, 906, 906, 843, 906, 168, 869, 168, 869, 843, 175,
        869, 843, 906, 869, 906, 906, 906, 843, 869, 906, 843, 906, 861, 906,
        906, 168, 869, 168, 843, 869, 843, 906, 843, 143, 168, 843, 843, 843,
        168, 659, 869, 143, 906, 869, 143, 859, 280, 143, 906, 906, 869, 168,
        843, 906, 843, 906, 143, 168, 869, 143, 843, 143, 143, 843, 143, 843,
        906, 869, 823, 843, 843, 143, 364, 143, 168, 906, 168, 906, 143, 843,
        906, 168, 843, 851, 869, 906, 168, 851, 843, 662, 168, 9

In [65]:
preds.shape

torch.Size([8, 80, 950])